In [1]:
import numpy as np
import copy
import torch
from transformers import logging
logging.set_verbosity_error()

from PIL import Image
#!pip install matplotlib
import matplotlib.pyplot as plt

from fromage_inf.fromage import models
from fromage_inf.fromage import utils

ModuleNotFoundError: No module named 'fromage'

In [ ]:
# For the LISA implementation, we probably should intercept the [RET] token outputs before retrieving them from the cc3m pickle, and save them
# Wait since we are doing from the notebook, maybe not
# But then we can't simply print all 30 prompts out with and without CoT
# would be too long and ugly of a notebook no?

# This file was used to try stuff out to get fromage to work
## For a better example of the code look at the FROMAGe notebook in fromage_inf

In [2]:
def trunc_caption(caption: str) -> str:
    # Truncate at period.
    trunc_index = caption.find('.') + 1
    if trunc_index < 0:
        trunc_index = caption.find('\n') + 1
    caption = caption[:trunc_index]
    return caption

def display_interleaved_outputs(model_outputs, one_img_per_ret=True):
    for output in model_outputs:
        if type(output) == str:
            print(output)
        elif type(output) == list:
            # Use this to display the single prompt image
            if one_img_per_ret:
                image = Image.open("../benchmark/" + output[0] + ".jpg")
                image = image.resize((224, 224))
                image = image.convert('RGB')
                display(image)
            # Use this to display the RET image/s
            else:
                fig, ax = plt.subplots(1, len(output), figsize=(3 * len(output), 3))
                for i, image in enumerate(output):
                    image = np.array(image)
                    ax[i].imshow(image)
                    ax[i].set_title(f'Retrieval #{i+1}')
            plt.show()

In [3]:
# Load model used in the paper.
model_dir = './fromage_inf/fromage_model/'
model = models.load_fromage(model_dir)

Using HuggingFace AutoFeatureExtractor for openai/clip-vit-large-patch14.
Using facebook/opt-6.7b for the language model.
Using openai/clip-vit-large-patch14 for the visual model with 1 visual tokens.


: 

: 

In [ ]:
# Load an image of a leaders/angela_merkel.
inp_image = ["leaders/angela_merkel"]

# Get FROMAGe to retrieve images of cats in other styles.
for inp_text in ['[RET]', 'at the beach [RET]', 'in a forest [RET]']:
    prompt = [inp_image, inp_text]
    print('Prompt:')
    display_interleaved_outputs(prompt)
    print('=' * 30)
    model_outputs = model.generate_for_images_and_texts(prompt, max_img_per_ret=3)

    # Display outputs.
    print('Model generated outputs:')
    display_interleaved_outputs(model_outputs, one_img_per_ret=False)